In [1]:
# from __future__ import division
import itertools

# To get rid of those blocks of red warnings
import warnings
warnings.filterwarnings("ignore")

# Standard Imports
import numpy as np
from scipy import stats
import pandas as pd
from math import sqrt
import os
from scipy.stats import spearmanr
from sklearn import metrics
from random import randint
import re

# Vis Imports
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import plotly.express as px
from pandas.plotting import register_matplotlib_converters
from mpl_toolkits.mplot3d import Axes3D

# Modeling Imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import sklearn.preprocessing
import statsmodels.api as sm
from sklearn.cluster import DBSCAN

# Custom Module Imports
import env

### Write a function named is_vowel. It should accept a string as input and use a regular expression to determine if the passed string is a vowel. While not explicity mentioned in the lesson, you can treat the result of re.search as a boolean value that indicates whether or not the regular expression matches the given string.

In [2]:
re.search(r'[aeiou\s\.]', 'abcd') # any letter that's a vowel


<re.Match object; span=(0, 1), match='a'>

In [3]:
def is_vowel(string):
    regex = r'[aeiou\s\.]'
    search = re.search(regex, string)
    boolean = bool(search)
    print(boolean)

In [4]:
string = 'Mary had a little lamb. 1 little lamb. Not 10, not 12, not 22, just one.'
is_vowel(string)


True


In [5]:
string = 'bcd'
is_vowel(string)


False


### Write a function named is_valid_username that accepts a string as input. A valid username starts with a lowercase letter, and only consists of lowercase letters, numbers, or the _ character. It should also be no longer than 32 characters. The function should return either True or False depending on whether the passed string is a valid username.

In [6]:
def is_valid_username(username):
    regex = r"[0–9a-z_]{1,32}$"
    boolean = bool(re.match(regex, username))
    print(boolean)

In [7]:
username = 'codeup'
is_valid_username(username)

True


In [8]:
username = 'Codeup'
is_valid_username(username)

False


### Write a regular expression to capture phone numbers. It should match all of the following:



(210) 867 5309

+1 210.867.5309

867-5309

210-867-5309

In [9]:
number = '(210) 867 5309'
regex = r'[0-9]+'
match = re.findall(regex, number)
''.join(match)

'2108675309'

In [10]:
number = '+1 210.867.5309'
regex = r'[0-9]+'
match = re.findall(regex, number)
''.join(match)

'12108675309'

In [11]:
number = '867-5309'
regex = r'[0-9]+'
match = re.findall(regex, number)
''.join(match)

'8675309'

In [12]:
number = '210-867-5309'
regex = r'[0-9]+'
match = re.findall(regex, number)
''.join(match)

'2108675309'

### Use regular expressions to convert the dates below to the standardized year-month-day format.




02/04/19

02/05/19

02/06/19

02/07/19

02/08/19

02/09/19

02/10/19


In [13]:
re.sub(r'(\d{2})/(\d{2})/(\d{2})', r'\3/\1/\2', '02/04/19')

'19/02/04'

### Write a regex to extract the various parts of these logfile lines:



GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58
POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58
GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58


In [14]:
logfile = ['GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58',
'POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58',
'GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58']

In [15]:
regexp = r'(?P<method>[A-Z]+)\s(?P<path>.+?)\s\[(?P<date>.+?)\]\s(?P<version>.+?)\s(?P<status>{\d+})\s(?P<bytes_sent>\d+)\s(?P<user_agent>".+?")\s(?P<ip>.+)'

for log in logfile:
    print(re.findall(regexp, log))

[('GET', '/api/v1/sales?page=86', '16/Apr/2019:193452+0000', 'HTTP/1.1', '{200}', '510348', '"python-requests/2.21.0"', '97.105.19.58')]
[('POST', '/users_accounts/file-upload', '16/Apr/2019:193452+0000', 'HTTP/1.1', '{201}', '42', '"User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36"', '97.105.19.58')]
[('GET', '/api/v1/items?page=3', '16/Apr/2019:193453+0000', 'HTTP/1.1', '{429}', '3561', '"python-requests/2.21.0"', '97.105.19.58')]


In [16]:
pattern = re.compile(r'''
        (?P<method>[A-Z]+)\s
        (?P<path>.+?)\s\[
        (?P<date>.+?)\]\s
        (?P<version>.+?)\s
        (?P<status>{\d+})\s
        (?P<bytes>\d+)\s"
        (?P<user_agent>.+?)"\s
        (?P<ip>.+)
        ''', re.VERBOSE)

pattern

re.compile(r'\n        (?P<method>[A-Z]+)\s\n        (?P<path>.+?)\s\[\n        (?P<date>.+?)\]\s\n        (?P<version>.+?)\s\n        (?P<status>{\d+})\s\n        (?P<bytes>\d+)\s"\n        (?P<user_agent>.+?)"\s\n        (?P<ip>.+)\n        ',
           re.UNICODE|re.VERBOSE)

In [17]:
contacts = [re.search(pattern, log).groupdict() for log in logfile]
contacts

[{'method': 'GET',
  'path': '/api/v1/sales?page=86',
  'date': '16/Apr/2019:193452+0000',
  'version': 'HTTP/1.1',
  'status': '{200}',
  'bytes': '510348',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'},
 {'method': 'POST',
  'path': '/users_accounts/file-upload',
  'date': '16/Apr/2019:193452+0000',
  'version': 'HTTP/1.1',
  'status': '{201}',
  'bytes': '42',
  'user_agent': 'User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
  'ip': '97.105.19.58'},
 {'method': 'GET',
  'path': '/api/v1/items?page=3',
  'date': '16/Apr/2019:193453+0000',
  'version': 'HTTP/1.1',
  'status': '{429}',
  'bytes': '3561',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'}]

In [18]:
logfile = pd.DataFrame(contacts)

In [19]:
logfile.head()

,method,path,date,version,status,bytes,user_agent,ip
0,GET,/api/v1/sales?page=86,16/Apr/2019:193452+0000,HTTP/1.1,{200},510348,python-requests/2.21.0,97.105.19.58
1,POST,/users_accounts/file-upload,16/Apr/2019:193452+0000,HTTP/1.1,{201},42,User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; ...,97.105.19.58
2,GET,/api/v1/items?page=3,16/Apr/2019:193453+0000,HTTP/1.1,{429},3561,python-requests/2.21.0,97.105.19.58
